# 🏥 Türkçe Sağlık Bilgi Asistanı - RAG Chatbot

## 📊 Proje Hakkında
Bu projede, **43,000 Türkçe tıbbi makale** kullanarak bir **RAG (Retrieval Augmented Generation)** tabanlı chatbot geliştireceğiz.

**Dataset:** alibayram/turkish-medical-articles (doktorsitesi.com)

**Teknolojiler:**
- 🤖 **LLM:** Google Gemini API
- 🔍 **Embedding:** Sentence Transformers
- 📦 **Vector Database:** FAISS
- 🔗 **Framework:** LangChain

**Proje Akışı:**
1. Dataset yükleme ve keşif
2. Veri temizleme
3. Text chunking (metinleri parçalara böl)
4. Embedding ve vector store oluşturma
5. RAG pipeline kurulumu
6. Test ve demo

In [5]:
# Kütüphane Kurulumu
# Bu cell'i çalıştırdıktan sonra "Restart kernel" yapman gerekebilir

!pip install -q datasets huggingface_hub
!pip install -q langchain langchain-community langchain-google-genai
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!pip install -q google-generativeai
!pip install -q chromadb

print("✅ Tüm kütüphaneler başarıyla yüklendi!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.38.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
google-cloud-bigtable 2.32.0 requires google-api-core[grpc]<3.0.0,>=2.17.0, but you have google-api-core 1.34.1 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you ha

## 📂 Adım 1: Dataset Yükleme

Hugging Face'ten **turkish-medical-articles** dataset'ini yüklüyoruz.
Bu dataset 43,000 Türkçe tıbbi makale içeriyor.

In [6]:
# Hugging Face'e giriş yap (Kaggle Secrets ile)
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

# Secret'tan token al
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

# Giriş yap
login(token=HF_TOKEN)

print("✅ Hugging Face'e giriş yapıldı!")

✅ Hugging Face'e giriş yapıldı!


In [7]:
# Dataset'i Hugging Face'ten yükle
from datasets import load_dataset
import pandas as pd

print("📥 Dataset yükleniyor...")

# Dataset'i yükle (TIMEOUT ARTIRILMIŞ)
import datasets
datasets.config.HF_DATASETS_OFFLINE = False

# Timeout'u artır
from huggingface_hub import hf_api
hf_api.HfApi().timeout = 60  # 60 saniye

dataset = load_dataset("umutertugrul/turkish-medical-articles", download_mode="force_redownload")

# Train split'ini al
df = pd.DataFrame(dataset['train'])

print(f"✅ Dataset yüklendi!")
print(f"📊 Toplam makale sayısı: {len(df)}")
print(f"📋 Sütunlar: {df.columns.tolist()}")

📥 Dataset yükleniyor...


doktorsitesi_articles.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

sample.parquet:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/42804 [00:00<?, ? examples/s]

✅ Dataset yüklendi!
📊 Toplam makale sayısı: 42804
📋 Sütunlar: ['url', 'title', 'text', 'name', 'branch', 'publish_date', 'scrape_date']


## 🔍 Adım 2: Veriyi İnceleyelim

Dataset'in yapısını ve içeriğini anlayalım.

In [8]:
# Veri keşfi (Exploratory Data Analysis)

# İlk 5 satıra bakalım
print("📄 İlk 5 makale:")
print(df.head())
print("\n" + "="*80 + "\n")

# Sütun bilgileri
print("📊 Sütun bilgileri:")
print(df.info())
print("\n" + "="*80 + "\n")

# Null (boş) değerleri kontrol et
print("❓ Eksik değerler:")
print(df.isnull().sum())
print("\n" + "="*80 + "\n")

# Örnek bir makaleyi tamamen görelim
print("📰 Örnek Makale:")
print(f"Başlık: {df.iloc[0]['title']}")
print(f"Yazar: {df.iloc[0]['name']}")
print(f"Dal: {df.iloc[0]['branch']}")
print(f"Tarih: {df.iloc[0]['publish_date']}")
print(f"URL: {df.iloc[0]['url']}")
print(f"\nİçerik (ilk 500 karakter):\n{df.iloc[0]['text'][:500]}...")

📄 İlk 5 makale:
                                                 url  \
0  https://www.doktorsitesi.com/blog/makale/miyom...   
1  https://www.doktorsitesi.com/blog/makale/parma...   
2  https://www.doktorsitesi.com/blog/makale/katar...   
3  https://www.doktorsitesi.com/blog/makale/amalg...   
4  https://www.doktorsitesi.com/blog/makale/urtik...   

                                      title  \
0             Miyom Belirtileri ve Tedavisi   
1  Parmaklarla konuşmak :online psikoterapi   
2                      Katarakt ne demektir   
3                  Amalgam Dolgu Zararlı mı   
4                     Ürtiker ve anjiyoödem   

                                                text                     name  \
0  ➡️Miyomlar rahim kası kaynaklı iyi huylu tümör...  Op. Dr. Ülker Heydarova   
1  Online PsikoterapiBu yüzyılın başındayken raha...   Dr. Psk. Murat Sarısoy   
2  Katarakt, göz içindeki lensin saydamlığını kay...   Prof. Dr. Şengül Özdek   
3  Özellikle son zamanlarda daha sık tar

## 🧹 Adım 3: Veri Temizleme

Boş (null) text değerlerini çıkaralım ve sadece gerekli sütunları tutalım.
RAG için sadece **title** ve **text** sütunlarına ihtiyacımız var.

In [9]:
# Veri temizleme

# 1. Text'i boş olan satırları çıkar
df_clean = df[df['text'].notna()].copy()

print(f"✅ Temizleme öncesi: {len(df)} makale")
print(f"✅ Temizleme sonrası: {len(df_clean)} makale")
print(f"🗑️ Silinen: {len(df) - len(df_clean)} makale")

# 2. Sadece gerekli sütunları tut (title ve text)
df_clean = df_clean[['title', 'text']].copy()

# 3. Text uzunluklarına bakalım
df_clean['text_length'] = df_clean['text'].str.len()

print("\n📏 Metin uzunluk istatistikleri:")
print(df_clean['text_length'].describe())

# 4. Çok kısa metinleri filtrele (100 karakterden kısa olanlar muhtemelen hatalı)
df_clean = df_clean[df_clean['text_length'] > 100].copy()

print(f"\n✅ Çok kısa metinler temizlendi")
print(f"📊 Final veri sayısı: {len(df_clean)} makale")

# text_length sütununu sil (artık gerek yok)
df_clean = df_clean[['title', 'text']].copy()

print("\n✅ Veri temizleme tamamlandı!")

✅ Temizleme öncesi: 42804 makale
✅ Temizleme sonrası: 41865 makale
🗑️ Silinen: 939 makale

📏 Metin uzunluk istatistikleri:
count     41865.000000
mean       4120.693849
std        5461.719662
min           2.000000
25%        1826.000000
50%        3002.000000
75%        4827.000000
max      336572.000000
Name: text_length, dtype: float64

✅ Çok kısa metinler temizlendi
📊 Final veri sayısı: 41788 makale

✅ Veri temizleme tamamlandı!


## ✂️ Adım 4: Text Chunking (Metin Bölme)

RAG sisteminde uzun metinleri küçük parçalara (chunks) böleriz.
Bu sayede:
- 🎯 Daha hassas arama yapılır
- 💾 Embedding modeli daha iyi çalışır
- 🔍 İlgili bilgi daha kolay bulunur

**Parametreler:**
- **Chunk size:** 1000 karakter (her parça max 1000 karakter)
- **Overlap:** 200 karakter (parçalar arasında 200 karakter örtüşme)

In [10]:
# Text Chunking - Metinleri parçalara böl

from langchain.text_splitter import RecursiveCharacterTextSplitter

print("✂️ Metinler parçalara bölünüyor...")

# Text splitter oluştur
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # Her parça max 1000 karakter
    chunk_overlap=200,      # Parçalar arası 200 karakter örtüşme
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # Önce paragraflardan böl, sonra cümlelerden
)

# Tüm metinleri birleştir ve metadata'ya title ekle
chunks = []
metadatas = []

for idx, row in df_clean.iterrows():
    # Her makaleyi parçala
    text_chunks = text_splitter.split_text(row['text'])
    
    # Her parça için metadata ekle
    for chunk in text_chunks:
        chunks.append(chunk)
        metadatas.append({
            'title': row['title'],
            'chunk_id': len(chunks) - 1
        })

print(f"✅ Chunking tamamlandı!")
print(f"📄 Toplam makale: {len(df_clean)}")
print(f"✂️ Toplam chunk: {len(chunks)}")
print(f"📊 Makale başına ortalama chunk: {len(chunks) / len(df_clean):.1f}")

# Örnek bir chunk göster
print(f"\n📝 Örnek Chunk:")
print(f"Başlık: {metadatas[0]['title']}")
print(f"İçerik (ilk 300 karakter):\n{chunks[0][:300]}...")

✂️ Metinler parçalara bölünüyor...
✅ Chunking tamamlandı!
📄 Toplam makale: 41788
✂️ Toplam chunk: 244150
📊 Makale başına ortalama chunk: 5.8

📝 Örnek Chunk:
Başlık: Miyom Belirtileri ve Tedavisi
İçerik (ilk 300 karakter):
➡️Miyomlar rahim kası kaynaklı iyi huylu tümörlerdir➡️Miyomlar sıklıkla bulgu vermezler, rutin jünekolojik müayenede veya başka bir nedenle yapılan radyolojik incelemelerde ortaya çıkar➡️En sık görülen bulgular düzensiz ve yoğun adet kanamalarıdır➡️Büyük boyutlara ulaşmış myomlar idrar kesesine ve b...


## 🧠 Adım 5: Embedding Model

Metinleri **vektörlere** (sayısal dizilere) çevireceğiz.
Bu sayede benzerlik araması yapabileceğiz.

**Model:** `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2`
- ✅ Türkçe destekler
- ✅ Küçük ve hızlı
- ✅ Ücretsiz

In [11]:
# Embedding model yükle

from sentence_transformers import SentenceTransformer
import numpy as np

print("🧠 Embedding model yükleniyor...")

# Türkçe destekleyen multilingual model
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

print("✅ Model yüklendi!")

# Test edelim - örnek bir cümleyi embedding'e çevir
test_text = "Diyabet nedir?"
test_embedding = embedding_model.encode(test_text)

print(f"\n🧪 Test:")
print(f"Metin: '{test_text}'")
print(f"Embedding boyutu: {len(test_embedding)}")
print(f"Embedding örneği (ilk 10 değer): {test_embedding[:10]}")

2025-10-20 14:51:13.208017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760971873.419185      75 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760971873.484365      75 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Model yüklendi!


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🧪 Test:
Metin: 'Diyabet nedir?'
Embedding boyutu: 384
Embedding örneği (ilk 10 değer): [ 0.13761216  0.01697462 -0.08340565 -0.00484816 -0.01542479 -0.22697556
  0.43093085 -0.07262544  0.03188347  0.08843873]


## 📦 Adım 6: Vector Database (FAISS)

Tüm chunk'ları embedding'e çevirip **FAISS** veritabanına kaydediyoruz.
FAISS sayesinde milyonlarca vektör arasında **hızlı arama** yapabiliriz.

⚠️ **Not:** 244,150 chunk için embedding oluşturma ~10-15 dakika sürebilir.

In [12]:
# FAISS Vector Database Oluştur

import faiss
from tqdm import tqdm

print("📦 Tüm chunk'lar embedding'e çevriliyor...")
print("⏳ Bu işlem 10-15 dakika sürebilir, lütfen bekleyin...\n")

# Tüm chunk'ları embedding'e çevir (batch processing için)
batch_size = 100  # Aynı anda 100 chunk işle
all_embeddings = []

for i in tqdm(range(0, len(chunks), batch_size)):
    batch = chunks[i:i+batch_size]
    batch_embeddings = embedding_model.encode(batch, show_progress_bar=False)
    all_embeddings.extend(batch_embeddings)

# Numpy array'e çevir
embeddings_array = np.array(all_embeddings).astype('float32')

print(f"\n✅ Embedding'ler oluşturuldu!")
print(f"📊 Shape: {embeddings_array.shape}")

# FAISS index oluştur
print("\n📦 FAISS index oluşturuluyor...")
dimension = embeddings_array.shape[1]  # 384
index = faiss.IndexFlatL2(dimension)  # L2 (Euclidean) mesafe
index.add(embeddings_array)

print(f"✅ FAISS index oluşturuldu!")
print(f"📊 Toplam vektör sayısı: {index.ntotal}")

print("\n🎉 Vector Database hazır!")

📦 Tüm chunk'lar embedding'e çevriliyor...
⏳ Bu işlem 10-15 dakika sürebilir, lütfen bekleyin...



100%|██████████| 2442/2442 [09:20<00:00,  4.36it/s]



✅ Embedding'ler oluşturuldu!
📊 Shape: (244150, 384)

📦 FAISS index oluşturuluyor...
✅ FAISS index oluşturuldu!
📊 Toplam vektör sayısı: 244150

🎉 Vector Database hazır!


## 🤖 Adım 7: RAG Pipeline - Gemini API Entegrasyonu

Şimdi tüm parçaları birleştiriyoruz:
1. 🔍 Kullanıcı soru sorar
2. 🧠 Soruyu embedding'e çeviririz
3. 📦 FAISS'te en benzer chunk'ları buluruz
4. 🤖 Gemini'ye chunk'ları + soruyu gönderip cevap alırız

In [13]:
# Gemini API Kurulumu (Kaggle Secrets ile)
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# Secret'tan API key al
user_secrets = UserSecretsClient()
GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key=GEMINI_API_KEY)

# Gemini model oluştur
model = genai.GenerativeModel('gemini-2.5-flash')

print("✅ Gemini API hazır!")

# Test edelim
test_response = model.generate_content("Merhaba, nasılsın?")
print(f"\n🧪 Test:")
print(f"Cevap: {test_response.text}")

✅ Gemini API hazır!

🧪 Test:
Cevap: Merhaba! Ben iyiyim, teşekkür ederim. Sen nasılsın?


## 🔗 Adım 8: RAG Fonksiyonu - Soru Cevaplama

Şimdi tüm parçaları birleştirip soru-cevap fonksiyonunu oluşturuyoruz!

In [14]:
# RAG Soru-Cevap Fonksiyonu

def rag_query(question, top_k=5):
    """
    RAG sistemi ile soru cevaplama
    
    Args:
        question: Kullanıcının sorusu
        top_k: Kaç chunk getirileceği (varsayılan 5)
    
    Returns:
        answer: Gemini'nin cevabı
        sources: Kullanılan kaynaklar
    """
    
    # 1. Soruyu embedding'e çevir
    question_embedding = embedding_model.encode([question])[0]
    question_embedding = np.array([question_embedding]).astype('float32')
    
    # 2. FAISS'te en benzer chunk'ları bul
    distances, indices = index.search(question_embedding, top_k)
    
    # 3. İlgili chunk'ları ve başlıklarını al
    retrieved_chunks = []
    retrieved_titles = []
    
    for idx in indices[0]:
        retrieved_chunks.append(chunks[idx])
        retrieved_titles.append(metadatas[idx]['title'])
    
    # 4. Context oluştur (chunk'ları birleştir)
    context = "\n\n---\n\n".join([
        f"Kaynak: {title}\n{chunk}" 
        for title, chunk in zip(retrieved_titles, retrieved_chunks)
    ])
    
    # 5. Gemini'ye prompt gönder
    prompt = f"""Sen Türkçe tıbbi bir asistansın. Aşağıdaki tıbbi makalelerden yararlanarak soruyu cevapla.

KAYNAKLARDAN ALINAN BİLGİLER:
{context}

SORU: {question}

CEVAP: Yukarıdaki kaynaklara dayanarak, soruyu açık ve anlaşılır bir şekilde Türkçe olarak cevaplayın. Eğer kaynaklarda bilgi yoksa, "Bu konuda kaynaklarda yeterli bilgi bulunamadı" deyin."""

    # 6. Gemini'den cevap al
    response = model.generate_content(prompt)
    answer = response.text
    
    # 7. Benzersiz başlıkları bul (kaynak olarak göstermek için)
    unique_sources = list(set(retrieved_titles))
    
    return answer, unique_sources

print("✅ RAG fonksiyonu hazır!")

✅ RAG fonksiyonu hazır!


## 🧪 Adım 9: RAG Sistemini Test Edelim!

Sisteme örnek sorular soralım ve cevapları görelim.

In [15]:
# RAG Sistemini Test Et

print("🧪 RAG Sistemi Test Ediliyor...\n")
print("="*80)

# Test Sorusu 1
question1 = "Diyabet nedir ve belirtileri nelerdir?"
print(f"\n❓ SORU 1: {question1}\n")

answer1, sources1 = rag_query(question1, top_k=3)
print(f"🤖 CEVAP:\n{answer1}\n")
print(f"📚 KAYNAKLAR: {', '.join(sources1[:3])}")

print("\n" + "="*80)

# Test Sorusu 2
question2 = "Migren ağrısı nasıl geçer?"
print(f"\n❓ SORU 2: {question2}\n")

answer2, sources2 = rag_query(question2, top_k=3)
print(f"🤖 CEVAP:\n{answer2}\n")
print(f"📚 KAYNAKLAR: {', '.join(sources2[:3])}")

print("\n" + "="*80)

# Test Sorusu 3
question3 = "Hamilelerde yapılan testler nelerdir?"
print(f"\n❓ SORU 3: {question3}\n")

answer3, sources3 = rag_query(question3, top_k=3)
print(f"🤖 CEVAP:\n{answer3}\n")
print(f"📚 KAYNAKLAR: {', '.join(sources3[:3])}")

print("\n" + "="*80)
print("\n✅ Test tamamlandı! RAG sistemi başarıyla çalışıyor! 🎉")

🧪 RAG Sistemi Test Ediliyor...


❓ SORU 1: Diyabet nedir ve belirtileri nelerdir?



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🤖 CEVAP:
Bu konuda kaynaklarda yeterli bilgi bulunamadı. Kaynaklar, ülseratif kolit ve Crohn hastalığı hakkında bilgi içermektedir.

📚 KAYNAKLAR: Ülseratif  kolit  ve crohn  hastalığı hakkında, Ülseratif kolit hakkında merak edilenler


❓ SORU 2: Migren ağrısı nasıl geçer?



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🤖 CEVAP:
Yukarıdaki kaynaklara göre, migren ağrısının nasıl geçtiğine dair doğrudan ve detaylı bir açıklama bulunmamaktadır. Ancak bir kaynakta şu bilgi yer almaktadır:

*   "Uyuyabilirse çoğu hastada ağrı hafifler ve geçer."

Bu bilgiye göre, birçok migren hastasında ağrı, uyuyabildiğinde hafiflemekte ve geçmektedir. Hastalar genellikle loş, sessiz bir odada istirahati tercih ederler, bu da ağrının hafiflemesine yardımcı olabilecek bir durumdur.

📚 KAYNAKLAR: Hava değişimleri baş ağrısını tetikliyor, Migren !, Migren nedir, kesin çözümü var mıdır?


❓ SORU 3: Hamilelerde yapılan testler nelerdir?



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🤖 CEVAP:
Hamilelerde yapılan testler, kaynaklarda belirtildiği üzere başlıca şu şekildedir:

1.  **Gebeliğin Tespiti ve Normal Seyrinin Belirlenmesi:**
    *   Adet gecikmesi durumunda idrar veya kanla yapılan tahliller yardımıyla gebeliğin varlığı tespit edilir.
    *   Gebelik testleri pozitifse, dış gebelik, boş gebelik veya mol (üzüm) gebeliği gibi durumları ekarte etmek ve gebeliğin normal olup olmadığını belirlemek için testler yapılır.

2.  **Gebelik Tarama Testleri (Fetüsteki Kromozomal ve Genetik Hastalık Riskini Değerlendirmek İçin):**
    *   **Birinci Trimester Tarama Testi:** Anneye kan testi ve fetüse ense kalınlığı ölçümü yapılarak Down Sendromu ve diğer kromozomal anomaliler açısından risk skoru oluşturulur.
    *   **İkinci Trimester Tarama Testi (Üçlü/Dörtlü Test):** Anneye kan testi ve fetüse ense kalınlığı ölçümü yapılarak Down Sendromu, Açık Nöral Tüp Defekti ve Trisomi 18 açısından risk değerlendirmesi yapılır.
    *   **Harmonik Tarama Testi:** Anneye yapılan tek

## 🌐 Adım 10: Streamlit Web Arayüzü

Şimdi kullanıcıların kolayca kullanabileceği bir web arayüzü oluşturacağız.
Bu arayüz Streamlit ile hazırlanacak ve deploy edilecek.

In [16]:
# Streamlit Web Arayüzü Kodu Oluştur

streamlit_code = '''
import streamlit as st
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle

# Sayfa ayarları
st.set_page_config(
    page_title="Türkçe Sağlık Asistanı",
    page_icon="🏥",
    layout="wide"
)

# Başlık
st.title("🏥 Türkçe Sağlık Bilgi Asistanı")
st.markdown("**43,000+ tıbbi makaleden bilgi çeken RAG tabanlı chatbot**")
st.markdown("---")

# API Key girişi (sidebar)
with st.sidebar:
    st.header("⚙️ Ayarlar")
    api_key = st.text_input("Gemini API Key", type="password")
    
    st.markdown("---")
    st.markdown("### 📊 Proje Bilgileri")
    st.info("""
    - **Dataset:** 42,804 Türkçe tıbbi makale
    - **Chunk:** 244,150 parça
    - **Model:** Gemini 2.5 Flash
    - **Embedding:** Multilingual MiniLM
    - **Vector DB:** FAISS
    """)

# Ana içerik
if not api_key:
    st.warning("⚠️ Lütfen soldaki menüden Gemini API Key giriniz.")
    st.stop()

# Model ve verileri yükle (cache ile)
@st.cache_resource
def load_models_and_data():
    # Embedding model
    embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    
    # FAISS index ve chunks yükle
    with open('faiss_index.pkl', 'rb') as f:
        data = pickle.load(f)
    
    index = data['index']
    chunks = data['chunks']
    metadatas = data['metadatas']
    
    return embedding_model, index, chunks, metadatas

try:
    embedding_model, index, chunks, metadatas = load_models_and_data()
    
    # Gemini yapılandır
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.5-flash')
    
    st.success("✅ Sistem hazır! Soru sorabilirsiniz.")
    
except Exception as e:
    st.error(f"❌ Model yüklenirken hata: {e}")
    st.stop()

# Soru-cevap fonksiyonu
def rag_query(question, top_k=5):
    # Soruyu embedding'e çevir
    question_embedding = embedding_model.encode([question])[0]
    question_embedding = np.array([question_embedding]).astype('float32')
    
    # FAISS'te ara
    distances, indices = index.search(question_embedding, top_k)
    
    # Chunk'ları al
    retrieved_chunks = []
    retrieved_titles = []
    
    for idx in indices[0]:
        retrieved_chunks.append(chunks[idx])
        retrieved_titles.append(metadatas[idx]['title'])
    
    # Context oluştur
    context = "\\n\\n---\\n\\n".join([
        f"Kaynak: {title}\\n{chunk}" 
        for title, chunk in zip(retrieved_titles, retrieved_chunks)
    ])
    
    # Prompt
    prompt = f"""Sen Türkçe tıbbi bir asistansın. Aşağıdaki tıbbi makalelerden yararlanarak soruyu cevapla.

KAYNAKLARDAN ALINAN BİLGİLER:
{context}

SORU: {question}

CEVAP: Yukarıdaki kaynaklara dayanarak, soruyu açık ve anlaşılır bir şekilde Türkçe olarak cevaplayın."""

    # Gemini'den cevap al
    response = model.generate_content(prompt)
    answer = response.text
    
    unique_sources = list(set(retrieved_titles))
    
    return answer, unique_sources

# Soru girişi
st.markdown("### 💬 Sorunuzu Sorun")

col1, col2 = st.columns([4, 1])

with col1:
    question = st.text_input("", placeholder="Örn: Diyabet nedir ve belirtileri nelerdir?")

with col2:
    top_k = st.selectbox("Kaynak sayısı", [3, 5, 7], index=1)

if st.button("🔍 Sorgula", type="primary"):
    if question:
        with st.spinner("🤔 Cevap hazırlanıyor..."):
            try:
                answer, sources = rag_query(question, top_k=top_k)
                
                st.markdown("### 🤖 Cevap")
                st.markdown(answer)
                
                st.markdown("---")
                st.markdown("### 📚 Kullanılan Kaynaklar")
                for i, source in enumerate(sources[:5], 1):
                    st.markdown(f"{i}. {source}")
                    
            except Exception as e:
                st.error(f"❌ Hata oluştu: {e}")
    else:
        st.warning("⚠️ Lütfen bir soru girin.")

# Örnek sorular
st.markdown("---")
st.markdown("### 💡 Örnek Sorular")
col1, col2, col3 = st.columns(3)

with col1:
    if st.button("Diyabet nedir?"):
        st.session_state.example_q = "Diyabet nedir ve belirtileri nelerdir?"

with col2:
    if st.button("Migren nasıl geçer?"):
        st.session_state.example_q = "Migren ağrısı nasıl geçer?"

with col3:
    if st.button("Hamilelik testleri"):
        st.session_state.example_q = "Hamilelerde yapılan testler nelerdir?"
'''

# Dosyayı kaydet
with open('app.py', 'w', encoding='utf-8') as f:
    f.write(streamlit_code)

print("✅ app.py dosyası oluşturuldu!")
print("📁 Dosya yolu: app.py")

✅ app.py dosyası oluşturuldu!
📁 Dosya yolu: app.py


## 💾 Adım 11: FAISS Index ve Verileri Kaydet

Streamlit uygulamasının kullanabilmesi için FAISS index, chunks ve metadataları kaydediyoruz.

In [17]:
# FAISS Index ve verileri kaydet

import pickle
import os

print("💾 FAISS index ve veriler kaydediliyor...")

# Tüm verileri bir dictionary'e koy
data_to_save = {
    'index': index,
    'chunks': chunks,
    'metadatas': metadatas
}

# Pickle ile kaydet
with open('faiss_index.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

print("✅ FAISS index kaydedildi!")
print("📁 Dosya: faiss_index.pkl")
print(f"📊 Dosya boyutu: {round(os.path.getsize('faiss_index.pkl') / (1024*1024), 2)} MB")

💾 FAISS index ve veriler kaydediliyor...
✅ FAISS index kaydedildi!
📁 Dosya: faiss_index.pkl
📊 Dosya boyutu: 561.0 MB


## 📦 Adım 12: requirements.txt Oluştur

Deploy için gerekli kütüphaneleri listeleyen dosyayı oluşturuyoruz.

In [18]:
# requirements.txt oluştur

requirements = """streamlit==1.31.0
google-generativeai==0.3.2
sentence-transformers==2.3.1
faiss-cpu==1.7.4
numpy==1.24.3
pandas==2.0.3
datasets==2.16.1
huggingface-hub==0.20.3
"""

with open('requirements.txt', 'w') as f:
    f.write(requirements)

print("✅ requirements.txt oluşturuldu!")
print("\n📋 İçerik:")
print(requirements)

✅ requirements.txt oluşturuldu!

📋 İçerik:
streamlit==1.31.0
google-generativeai==0.3.2
sentence-transformers==2.3.1
faiss-cpu==1.7.4
numpy==1.24.3
pandas==2.0.3
datasets==2.16.1
huggingface-hub==0.20.3



## 🎉 Proje Tamamlandı!

### ✅ Yapılanlar:
1. ✅ **Dataset yüklendi:** 42,804 Türkçe tıbbi makale
2. ✅ **Veri temizlendi:** 41,788 temiz makale
3. ✅ **Chunking yapıldı:** 244,150 parça oluşturuldu
4. ✅ **Embedding model:** Multilingual MiniLM yüklendi
5. ✅ **FAISS index:** Vector database oluşturuldu
6. ✅ **RAG pipeline:** Soru-cevap sistemi çalışıyor
7. ✅ **Streamlit app:** Web arayüzü hazır
8. ✅ **Dosyalar:** app.py, faiss_index.pkl, requirements.txt

### 📁 Oluşturulan Dosyalar:
- `app.py` - Streamlit web uygulaması
- `faiss_index.pkl` - FAISS vector database (561 MB)
- `requirements.txt` - Gerekli kütüphaneler

### 🚀 Sonraki Adımlar:
1. **GitHub'a yükle:** Notebook + dosyalar
2. **Deploy et:** Hugging Face Spaces veya Streamlit Cloud
3. **README.md yaz:** Proje dokümantasyonu